In [128]:
import time
import logging
from neo4j import GraphDatabase
from neo4j import Record
import neo4j

In [162]:
from settings import URI, AUTH

In [ ]:
class App:

    def __init__(self, uri, authentication):
        self.driver = GraphDatabase.driver(uri, auth=authentication)

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def get_word_result(self, word):
        with self.driver.session() as session:
            result = session.execute_read(self._find_and_return_word_result, word)
            word_result = [record for record in result]
            graph_tuples = self.parse_records(word_result)
            return graph_tuples
    
    def parse_records(self, query_record: list[Record]) -> list[tuple]:
        word_records = [word_record.data()['p'] for word_record in query_record]
        graph_tuples = [(word_data[-3]['value'], word_data[-1]['value']) for word_data in word_records]
        return graph_tuples

    @staticmethod
    def _find_and_return_word_result(tx, word):
        query = (
            "MATCH (w:Word {value:$word}) MATCH p = (w)-[:HAS_WORD*1..3]->(:Word) RETURN nodes(p);"
        )
        result = tx.run(query, word=word)
        return [record for record in result]
    

In [158]:
class Neo4jManager:
    
    def __init__(self, uri: str, authentication: tuple[str, str]):
        self.driver = GraphDatabase.driver(uri, auth=authentication)
        self.session = self.driver.session(database='neo4j', default_access_mode=neo4j.READ_ACCESS)
        self.tx = self.session.begin_transaction()
        
    def reset_transaction(self):
        '''Close the Transaction if not already closed, and begin a new transaction'''
        if not self.tx.closed(): self.tx.close()
        self.tx = self.session.begin_transaction()
    
    def close(self):
        '''Close the Transaction, Session, and Driver Connections'''
        self.tx.close()
        self.session.close()
        self.driver.close()
    
    def get_word_result(self, word: str) -> list[tuple[str, str]]:
        sub_start = time.time()
        result = self.tx.run("MATCH (w:Word) USING TEXT INDEX w:Word(value) WHERE w.value = 'defenestration' MATCH p = (w)-[:HAS_WORD*1..3]->(:Word) RETURN p;")
        print(time.time() - sub_start)
        word_result = [record for record in result]
        graph_tuples = self.parse_records(word_result)
        return graph_tuples
    
    def parse_records(self, query_record: list[Record]) -> list[tuple[str, str]]:
        word_records = [word_record.data()['p'] for word_record in query_record]
        graph_tuples = [(word_data[-3]['value'], word_data[-1]['value']) for word_data in word_records]
        return graph_tuples

In [163]:
database = Neo4jManager(URI, AUTH)

In [164]:
start = time.time()
word_result = database.get_word_result("defenestration")
print(time.time() - start)
print(len(word_result))

0.0010008811950683594
5.5391833782196045
41586


In [165]:
database.close()

In [204]:
def parse_records(query_record: list[Record]) -> list[tuple]:
    word_records = [word_record.data()['p'] for word_record in query_record]
    graph_tuples = [(word_data[-3]['value'], word_data[-1]['value']) for word_data in word_records]
    return graph_tuples

def get_word_result(tx, word):
    word_result = []
    result = tx.run("MATCH (w:Word) USING TEXT INDEX w:Word(value) WHERE w.value = 'defenestration' MATCH p = (w)-[:HAS_WORD]->(:Word) RETURN p;") #4.16
    #print(result)
    #result = tx.run("MATCH (w:Word {value:'defenestration'}) MATCH p = (w)-[:HAS_WORD*1..2]->(:Word) RETURN nodes(p);") # 4.16
    #result = tx.run("MATCH (w:Word {value:'defenestration'}) MATCH p = (w)-[:HAS_WORD*1..2]->(:Word) RETURN p;") #4.19
    #word_result = [record for record in result]
    #print(word_result)
    #graph_tuples = parse_records(word_result)
    pre_data = time.time()
    data = result.graph()
    print(time.time() - pre_data)
    return data

In [205]:
start = time.time()
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session(database='neo4j', default_access_mode=neo4j.READ_ACCESS) as session:
        with session.begin_transaction() as tx:
            result = get_word_result(tx, "defenestration")
print(time.time() - start)

0.0010008811950683594
4.114757537841797


In [208]:
result

In [187]:
list(result.nodes)

[<Node element_id='265787' labels=frozenset({'Word'}) properties={'value': 'defenestration'}>,
 <Node element_id='257674' labels=frozenset({'Word'}) properties={'value': 'throw'}>,
 <Node element_id='208335' labels=frozenset({'Word'}) properties={'value': 'swift'}>,
 <Node element_id='138219' labels=frozenset({'Word'}) properties={'value': 'a'}>,
 <Node element_id='148424' labels=frozenset({'Word'}) properties={'value': 'dismissal'}>,
 <Node element_id='139673' labels=frozenset({'Word'}) properties={'value': 'or'}>,
 <Node element_id='144993' labels=frozenset({'Word'}) properties={'value': 'as'}>,
 <Node element_id='170844' labels=frozenset({'Word'}) properties={'value': 'person'}>,
 <Node element_id='169684' labels=frozenset({'Word'}) properties={'value': 'of'}>,
 <Node element_id='262681' labels=frozenset({'Word'}) properties={'value': 'expulsion'}>,
 <Node element_id='239699' labels=frozenset({'Word'}) properties={'value': 'window'}>,
 <Node element_id='243995' labels=frozenset({'Wo

In [188]:
list(result.relationships)

[<Relationship element_id='781043' nodes=(<Node element_id='265787' labels=frozenset({'Word'}) properties={'value': 'defenestration'}>, <Node element_id='257674' labels=frozenset({'Word'}) properties={'value': 'throw'}>) type='HAS_WORD' properties={}>,
 <Relationship element_id='781051' nodes=(<Node element_id='265787' labels=frozenset({'Word'}) properties={'value': 'defenestration'}>, <Node element_id='208335' labels=frozenset({'Word'}) properties={'value': 'swift'}>) type='HAS_WORD' properties={}>,
 <Relationship element_id='781042' nodes=(<Node element_id='265787' labels=frozenset({'Word'}) properties={'value': 'defenestration'}>, <Node element_id='138219' labels=frozenset({'Word'}) properties={'value': 'a'}>) type='HAS_WORD' properties={}>,
 <Relationship element_id='781052' nodes=(<Node element_id='265787' labels=frozenset({'Word'}) properties={'value': 'defenestration'}>, <Node element_id='148424' labels=frozenset({'Word'}) properties={'value': 'dismissal'}>) type='HAS_WORD' prop

In [130]:
start = time.time()
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session(database='neo4j', default_access_mode=neo4j.READ_ACCESS) as session:
        sub_start = time.time()
        word_graph = session.execute_read(get_word_result, "defenestration")
        print(word_graph)
        print(time.time() - sub_start)
print(time.time() - start)

Sub sub0.0005021095275878906
[<Record nodes(p)=[<Node element_id='265787' labels=frozenset({'Word'}) properties={'value': 'defenestration'}>, <Node element_id='187219' labels=frozenset({'Word'}) properties={'value': 'thing'}>]>, <Record nodes(p)=[<Node element_id='265787' labels=frozenset({'Word'}) properties={'value': 'defenestration'}>, <Node element_id='187219' labels=frozenset({'Word'}) properties={'value': 'thing'}>, <Node element_id='269534' labels=frozenset({'Word'}) properties={'value': 'fashionable'}>]>, <Record nodes(p)=[<Node element_id='265787' labels=frozenset({'Word'}) properties={'value': 'defenestration'}>, <Node element_id='187219' labels=frozenset({'Word'}) properties={'value': 'thing'}>, <Node element_id='280150' labels=frozenset({'Word'}) properties={'value': 'mild'}>]>, <Record nodes(p)=[<Node element_id='265787' labels=frozenset({'Word'}) properties={'value': 'defenestration'}>, <Node element_id='187219' labels=frozenset({'Word'}) properties={'value': 'thing'}>, <

In [169]:
word_graph.graph()

AttributeError: 'list' object has no attribute 'graph'

In [94]:
print(word_graph[120])

<Record nodes(p)=[<Node element_id='265787' labels=frozenset({'Word'}) properties={'value': 'defenestration'}>, <Node element_id='207396' labels=frozenset({'Word'}) properties={'value': 'office'}>, <Node element_id='145128' labels=frozenset({'Word'}) properties={'value': 'for'}>]>


In [10]:
type(word_graph[0])

neo4j._data.Record

In [18]:
word_graph[0]

<Record p=<Path start=<Node element_id='265787' labels=frozenset({'Word'}) properties={'value': 'defenestration'}> end=<Node element_id='187219' labels=frozenset({'Word'}) properties={'value': 'thing'}> size=1>>

In [32]:
word_graph[0].data()['p']

[{'value': 'defenestration'}, 'HAS_WORD', {'value': 'thing'}]

In [34]:
len(word_graph)

890

In [39]:
for word_rel in word_graph:
    word_data = word_rel.data()['p']
    print((word_data[-3]['value'], word_data[-1]['value']))
    #print(word_data)

('defenestration', 'thing')
('thing', 'fashionable')
('thing', 'mild')
('thing', 'dressing')
('thing', 'possession')
('thing', 'act')
('thing', 'something')
('thing', 'product')
('thing', 'usually')
('thing', 'material')
('thing', 'observation')
('thing', 'imply')
('thing', 'not')
('thing', 'being')
('thing', 'living')
('thing', 'talk')
('thing', 'appeal')
('thing', 'precisely')
('thing', 'situation')
('thing', 'proper')
('thing', 'separate')
('thing', 'specialty')
('thing', 'general')
('thing', 'from')
('thing', 'article')
('thing', 'to')
('thing', 'write')
('thing', 'designate')
('thing', 'accomplishment')
('thing', 'activity')
('thing', 'of')
('thing', 'an')
('thing', 'distinguished')
('thing', 'concern')
('thing', 'news')
('thing', 'appearance')
('thing', 'spatial')
('thing', 'obsession')
('thing', 'the')
('thing', 'make')
('thing', 'entity')
('thing', 'own')
('thing', 'concrete')
('thing', 'circumstance')
('thing', 'utensil')
('thing', 'clothing')
('thing', 'spoken')
('thing', 'as

In [45]:
def parse_records(query_record: list[Record]) -> list[tuple]:
    word_records = [word_record.data()['p'] for word_record in query_record]
    graph_tuples = [(word_data[-3]['value'], word_data[-1]['value']) for word_data in word_records]
    return graph_tuples

In [58]:
start = time.time()
parse_records(word_graph)
print(time.time() - start)

0.012511014938354492
